# Training 

Notebooks to train models  
  


In [ ]:
# CONFG CELL FOR  PRVATE GIT ON COLAB
# ! sudo apt-get update
# ! sudo apt-get install --only-upgrade libssl-dev
# ! ssh-agent
# ! ssh-add /root/.ssh/id_rsa

! mkdir -p /root/.ssh
from pathlib import Path
import shutil
if not Path('/content/drive').exists():
  from google.colab import drive
  drive.mount('/content/drive')
shutil.copyfile('/content/drive/MyDrive/Projects/Forecast/id_rsa', '/root/.ssh/id_rsa')

! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
! chmod go-rwx /root/.ssh/id_rsa
if Path('/content/sundl').exists():
  shutil.rmtree('/content/sundl', ignore_errors=True)
! git clone git@github.com:gfrancisco20/sundl-private.git
!mv /content/sundl-private /content/sundl
%cd /content/sundl-private

# Config

In [1]:
COLAB = False


if COLAB : 
  configSetup = {
      'COLAB'           : 'True',
      'PATH_ROOT_DRIVE' : '/content/drive/MyDrive/Projects/Forecast',
      'PATH_ROOT_LOCAL' : '/content/session',
      'PATH_SUNDL'      : '/content/sundl',
      'PATH_PROJECT'    : '/content/sundl/notebooks/ficat'
  }
  !git clone https://github.com/gfrancisco20/sundl-private.git
  import sys
  import re
  sys.path.append(configSetup['PATH_SUNDL'])
  sys.path.append(configSetup['PATH_PROJECT'])
  configFile = f'{configSetup["PATH_PROJECT"]}/config.py'
  with open(configFile, 'r') as file:
    content = file.read()
  for constant in configSetup.keys():
    content = re.sub(re.compile(f'{constant} = .*'), f'{constant} = \'{configSetup[constant]}\'', content)
  with open(configFile, 'w') as file:
    file.write(content)
   
from config import *
from sundl.utils.colab import mountDrive
if COLAB:
  # mouting drive content in session on colab
  mountDrive()

2024-02-07 18:12:04.320199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Libraries

In [2]:
import shutil
from pathlib import Path
import dill as pickle

import time
from tqdm import tqdm
import gc

import datetime
import numpy as np
import pandas as pd

import tensorflow as tf

# Data import

In [3]:
%%time
from sundl.utils.colab import mountDrive, ressourcesSetAndCheck, drive2local
############################
# SETUP
############################

# overwriting CLEAN_LOCAL :
CLEAN_LOCAL = False

if CLEAN_LOCAL:
  shutil.rmtree(PATH_ROOT_LOCAL)
  os.makedirs(PATH_ROOT_LOCAL)
  
# checking gpu and ram ressources
ressourcesSetAndCheck(MIXED_PREC)

############################
# DATA IMPORT
############################

FILES2TRANSFER = {'images' : (PATH_ROOT_DRIVE_DS/'Images',        # source
                              PATH_IMAGES,                        # dest
                              ['ce_193x211x94_224'] # files
                              )
                  }

drive2local(FILES2TRANSFER)


Not connected to a GPU
Your runtime has 17.2 gigabytes of available RAM

Not using a high-RAM runtime

images files already in local
CPU times: user 1.88 ms, sys: 2.05 ms, total: 3.93 ms
Wall time: 13.1 ms


sh: nvidia-smi: command not found


# Hyperparameters

In [4]:


ts_off_label_days  = np.arange(0.5,6.5,0.5)   # lag with respect to sample/forecast date (the image date is the sample/forecast date)
ts_off_scalar_days = None


labelCol      = 'sw_v' # 'mpf' -> windows's SXR-max-peak-flux , 'toteh' -> (hourly average of sum of flares' SXR-fluence)
windows_avg_h = 12
EPOCHS        = 2 # 25
BATCH_SIZE    = 16
IMG_SIZE      = (224, 224, 3) # (512, 1024, 3) (224, 448, 3)
# PTCH_SIZE    = (112, 112, 3) # (256, 256, 3) (112, 112, 3)

NEW_FOLDER_NAME   = 'Tests'                 # New folder in which to store results
CONTINUING_FOLDER = None # Existing foler in which to store results

# CV PARAMETERS
CV_K          = 1 
VAL_SPLIT     = None # --> not used if CV_K not none
testSplitDate = datetime.datetime(2019,1,1) # splitting date between CV and operational tes
buffer        = pd.DateOffset(days=4)      # size of discarded buffer between temporal chunks (ensure folds indepedancy)
chunk_width   = pd.DateOffset(days=20)      # size of temporal chunks
bufferTest    = pd.DateOffset(days=27)
bufferCme     = pd.DateOffset(days=4)  

SAMPLE_TRAIN = 0.01 # 0.95 
SAMPLE_VAL   = 0.05

CACHE     = True
verbose   = 0

SAVE_MODEL   = True
save_monitor = 'val_rmse'
save_mode    = 'min'
save_thd     = 85
RECOMPUTE_DATASET = True


metrics = [tf.keras.metrics.MeanAbsoluteError('mae'),
           tf.keras.metrics.MeanAbsolutePercentageError('maep'),
           tf.keras.metrics.RootMeanSquaredError('rmse')
          ]
          #  tf.keras.metrics.R2Score(name='r2',]
   
ts_off_label_hours = [int(24*off) for off in ts_off_label_days]   # lag with respect to sample/forecast date (the image date is the sample/forecast date)
if ts_off_scalar_days is not None:
  ts_off_scalar_hours = [int(24*off) for off in ts_off_scalar_days] 
else:
  ts_off_scalar_hours = None

# Models Definition

In [5]:
from utilsTraining import ModelInstantier2
from sundl.models.blueprints import build_pretrained_model, build_persistant_model
from sundl.dataloader.sdocml import builDS_image_feature, buildDS_persistant_MTS
from tensorflow.keras.optimizers import Adam, AdamW

loss = tf.keras.losses.MeanSquaredError(name = 'loss')

optimizer = AdamW(
    learning_rate = 1e-3,
    weight_decay  = 1e-3,
    amsgrad       = True
)

spatial_feature_red = tf.keras.layers.Conv2D(name = 'top_conv',
                                  filters = 1,
                                  kernel_size = (4,4),
                                  activation = None
                                  )

ts_off_scalar_hours = None
if ts_off_scalar_hours is None:
  scalarFeaturesSize = None
else:
  scalarFeaturesSize = len(ts_off_scalar_hours)
  


sp16 = ModelInstantier2(
    buildModelFunction = build_pretrained_model,
    buildModelParams = {
        'tfModel'   : tf.keras.applications.efficientnet_v2.EfficientNetV2S,
        'metrics'   : metrics,
        'optimizer' : optimizer,
        'loss'      : loss,
        'img_size'  : IMG_SIZE,
        'regression': True,
        'lastTfConv'        : 'top_conv',
        'feature_reduction' : spatial_feature_red,
        'globalPooling'     : False,
        'pretainedWeight' : True,
        'unfreeze_top_N' : None,
        'scalarFeaturesSize' : None,
        'labelSize'          : len(ts_off_label_hours)
        },
    buildDsFunction = builDS_image_feature,
    buildDsParams  = {
        'img_size'    : None,
        'labelCol'    : labelCol,
        'regression'  : True,
        'ts_off_label_hours' : ts_off_label_hours,
        'ts_off_scalar_hours': None,
        },
    name       = 'SW',
    archiTag   = 'EfN2S',
    featureTag = 'SpX16', # @['Sp' -> spatial features , 'Gl' -> avg-pooled feeatures]   
)

sp16scFt = ModelInstantier2(
    buildModelFunction = build_pretrained_model,
    buildModelParams = {
        'tfModel'   : tf.keras.applications.efficientnet_v2.EfficientNetV2S,
        'metrics'   : metrics,
        'optimizer' : optimizer,
        'loss'      : loss,
        'img_size'  : IMG_SIZE,
        'regression': True,
        'lastTfConv'        : 'top_conv',
        'feature_reduction' : spatial_feature_red,
        'globalPooling'     : False,
        'pretainedWeight' : True,
        'unfreeze_top_N' : None,
        'scalarAgregation'   : 'feature',
        'scalarFeaturesSize' : len(ts_off_label_hours),
        'labelSize'          : len(ts_off_label_hours)
        },
    buildDsFunction = builDS_image_feature,
    buildDsParams  = {
        'img_size'    : None,
        'labelCol'    : labelCol,
        'regression'  : True,
        'ts_off_label_hours' : ts_off_label_hours,
        'ts_off_scalar_hours': [offlabel - 27*24 for offlabel in ts_off_label_hours],
        },
    name       = 'SW',
    archiTag   = 'EfN2S',
    featureTag = 'SpX16', # @['Sp' -> spatial features , 'Gl' -> avg-pooled feeatures]   
)

sp16scBs = ModelInstantier2(
    buildModelFunction = build_pretrained_model,
    buildModelParams = {
        'tfModel'   : tf.keras.applications.efficientnet_v2.EfficientNetV2S,
        'metrics'   : metrics,
        'optimizer' : optimizer,
        'loss'      : loss,
        'img_size'  : IMG_SIZE,
        'regression': True,
        'lastTfConv'        : 'top_conv',
        'feature_reduction' : spatial_feature_red,
        'globalPooling'     : False,
        'pretainedWeight' : True,
        'unfreeze_top_N' : None,
        'scalarAgregation'   : 'baseline',
        'scalarFeaturesSize' : len(ts_off_label_hours),
        'labelSize'          : len(ts_off_label_hours)
        },
    buildDsFunction = builDS_image_feature,
    buildDsParams  = {
        'img_size'    : None,
        'labelCol'    : labelCol,
        'regression'  : True,
        'ts_off_label_hours' : ts_off_label_hours,
        'ts_off_scalar_hours': [offlabel - 27*24 for offlabel in ts_off_label_hours],
        },
    name       = 'SW',
    archiTag   = 'EfN2S',
    featureTag = 'SpX16', # @['Sp' -> spatial features , 'Gl' -> avg-pooled feeatures]   
)


persistant27 = ModelInstantier2(
    buildModelFunction = build_persistant_model,
    buildModelParams = {
        'metrics'   : metrics,
        'optimizer' : optimizer,
        'loss'      : loss,
        'regression': True
        },
    buildDsFunction = buildDS_persistant_MTS,
    buildDsParams  = {
        'labelCol'    : labelCol,
        'regression'  : True,
        'ts_off_label_hours'  : ts_off_label_hours,
        'ts_off_history_hours': [offlabel - 27*24 for offlabel in ts_off_label_hours],
        },
    name       = 'SW',
    archiTag   = 'Pst27',
    featureTag = '', # @['Sp' -> spatial features , 'Gl' -> avg-pooled feeatures]   
)

persistant4 = ModelInstantier2(
    buildModelFunction = build_persistant_model,
    buildModelParams = {
        'metrics'   : metrics,
        'optimizer' : optimizer,
        'loss'      : loss,
        'regression': True
        },
    buildDsFunction = buildDS_persistant_MTS,
    buildDsParams  = {
        'labelCol'    : labelCol,
        'regression'  : True,
        'ts_off_label_hours'  : ts_off_label_hours,
        'ts_off_history_hours': [offlabel - 4*24 for offlabel in ts_off_label_hours],
        },
    name       = 'SW',
    archiTag   = 'Pst04',
    featureTag = '', # @['Sp' -> spatial features , 'Gl' -> avg-pooled feeatures]   
)

# Training

In [6]:
from utilsTraining import setUpResultFolder, trainConstantModel, printTrainingResults, saveTrainingResults
from sundl.utils.data import read_Dataframe_With_Dates, loadMinMaxDates

models = [
  (persistant27, None),
  (persistant4 , None),
  (sp16        , ['0193x0211x0094']),
  (sp16scFt    , ['0193x0211x0094']),
  (sp16scBs    , ['0193x0211x0094'])
]

log, resDir, modelDir, mtcDict = setUpResultFolder(
    models = models, 
    pathRes = PATH_RES,
    metrics = metrics,
    continuingFolder = CONTINUING_FOLDER, 
    newFolder = NEW_FOLDER_NAME,
    imgSize = IMG_SIZE,
    cv_K = CV_K,
    saveModel = SAVE_MODEL,
    windows_avg_h = windows_avg_h
    )

print('\nINITIAL STATUS : ')
display(log)
print('')
res = {}
best = None
eval = None
bestCVCrossEpoch = None
dsTrain = None
dsVal = None
ct=0
verbose = 1
ct_dsBuilds = -1
minDate, maxDate = loadMinMaxDates(PATH_IMAGES)

print('minDate : ', minDate)
print('maxDate : ', maxDate)

for modelInstantiater, channels in tqdm(models):
  h = windows_avg_h
  ct_dsBuilds+=1
  
  modelInstantiater.buildDsParams['classTresholds'] = None
  modelInstantiater.buildDsParams['classWeights'] = None
  

  dfTimeseries = read_Dataframe_With_Dates(F_PATH_SW_TS(h))

  CV_FLD_PTH  = F_PATH_FOLDS_SW(chunk_width, buffer, testSplitDate, h)
  with open(CV_FLD_PTH, 'rb') as f1:
    dfFoldsTrainVal = pickle.load(f1)[0:CV_K]
    
  full_name_comb = modelInstantiater.fullNameFunc(channels,h)
  if log.loc[full_name_comb]['status'] > 0:
    print(f'\n\n-----------------------------\n{full_name_comb} already successfuly trained\n')
  else:
    log.loc[full_name_comb, 'status'] = -1
    log.to_csv(resDir + '/log.csv')
  
    model = None
    if RECOMPUTE_DATASET:
      dsTrain = None
      dsVal = None
  
    #===================================================
    # CROSS VALIDATION LOOP
    #===================================================
    duration = time.time()
    res[full_name_comb] = []
    kf=0
    for df_train, df_val in tqdm(dfFoldsTrainVal,disable = False):#not verbose):
      print(f'\n\n-----------------------------\nModel : {full_name_comb}')
      if Path(resDir+f'/training_folds/training_{full_name_comb}_fd{kf:0>3d}.csv').exists():
        res[full_name_comb].append(pd.read_csv(resDir+f'/training_folds/training_{full_name_comb}_fd{kf:0>3d}.csv').set_index('epoch'))
        print(f'FOLD #{kf} ALREADY TRAINED')
        kf += 1
      else:
        print(f'\n\n-----------------------------\nModel : {full_name_comb}')
        print(f'FOLD #{kf}')
        
        # MEMORY CLEANING
        tf.keras.backend.clear_session()
        tf.compat.v1.reset_default_graph()
        if model is not None: del model
        if RECOMPUTE_DATASET:
          if dsTrain is not None: del dsTrain
          if dsVal is not None: del dsVal
        gc.collect()
        
        # FOLDER FOR MODEL
        if SAVE_MODEL:
          modelDirSub = modelDir + f'/{full_name_comb}'
          if CV_K is not None:
            modelDirSub = modelDirSub + f'_fd{kf:0>3d}'
            
        # DATASETS INSTANTIATION 
        dfSamples_train = df_train.copy()
        dfSamples_val = df_val.copy()
        if SAMPLE_TRAIN is not None:
          dfSamples_train = dfSamples_train.sample(frac = SAMPLE_TRAIN, random_state=49)
        if SAMPLE_VAL is not None:
           dfSamples_val = dfSamples_val.sample(frac = SAMPLE_VAL, random_state=49)
        if ct_dsBuilds==0 or RECOMPUTE_DATASET:
          pathDir = PATH_IMAGES if channels is not None else None
          print('\nDS TRAINING : ')
          dsTrain, _, missing_file_regexp, dfSamples_train_corr = modelInstantiater.build_DS(
              pathDir    = pathDir,
              channels   = channels,
              dfTimeseries = dfTimeseries.copy(), 
              samples    = dfSamples_train.copy(), 
              batch_size = BATCH_SIZE,
              cache      = CACHE,
              shuffle    = True
          )
          print('\nDS VALIDATION : ')
          dsVal, _, missing_file_regexp_val, dfSamples_val_corr  = modelInstantiater.build_DS(
              pathDir    = pathDir,
              channels   = channels,
              dfTimeseries = dfTimeseries.copy(),
              samples    = dfSamples_val.copy(),
              batch_size = BATCH_SIZE,
              cache      = CACHE,
              shuffle    = True,
              typeDs     = 'val'
          )
          print(f'{len(missing_file_regexp)} incomplete training dates')
          print(f'{len(missing_file_regexp_val)} incomplete val dates')
          
        # MODEL INSTANTIATION
        model = modelInstantiater()
        try:
          print(f'\nMODEL PARAMETERS #: {model.count_params()/1e6:.2f}M')
          trainable_params = tf.reduce_sum([tf.reduce_prod(p.shape) for p in model.trainable_variables])
          print("of which trainable #:", trainable_params)
        except:
          pass
        
        # CALLBACKS
        callbacks = []
        if SAVE_MODEL:
          callbacks.append(tf.keras.callbacks.ModelCheckpoint(
              modelDirSub,
              save_best_only = True,
              save_weights_only=False,
              monitor = save_monitor,
              verbose = 1,
              mode = save_mode,
              initial_value_threshold = save_thd)
          )
        
        # TRAINING
        if modelInstantiater.savedPredictionModel:
          # models where scalar = output (e.g. persistant models)
          historyData = trainConstantModel(dsTrain, dsVal, model, modelInstantiater, EPOCHS, weightByClass, SAVE_MODEL, modelDirSub)
        else:
          history = model.fit(dsTrain,#.take(1),
                              epochs=EPOCHS,
                              validation_data = dsVal,#.take(1),
                              callbacks = callbacks,
                              verbose = 1 #verbose
                              )
          historyData = history.history
          if SAVE_MODEL:
            pathConfigModel = modelDirSub + f'_config.pkl'
            modelInstantiater.saveConfig(pathConfigModel)

        # PRINTING TRAINING HISTORY
        printTrainingResults(historyData)
        

        # SAVING FOLD RESULTS
        res[full_name_comb].append(pd.DataFrame(historyData)) 
        res[full_name_comb][-1].index.names = ['epoch']
        num_inst = len(dfSamples_train)
        res[full_name_comb][-1]['num_train_inst'] = (res[full_name_comb][-1].index + 1) * num_inst
        res[full_name_comb][-1].to_csv(resDir+f'/training_folds/training_{full_name_comb}_fd{kf:0>3d}.csv',index=True)
        kf+=1 # fold index
       
      
      # END OF CV-LOOP
      #===================================================
      
    # SAVING GENERAL RESULTS
    res, best, bestCVCrossEpoch = saveTrainingResults(resDir, res, best, bestCVCrossEpoch, full_name_comb, CV_K)
    
    duration = time.time() - duration
    log.loc[full_name_comb, 'status'] = 1
    log.loc[full_name_comb, 'duration'] = f'{duration//3600:0>2.0f}h {duration//60%60:0>2.0f}m {duration%60:0>2.0f}s'
    log.to_csv(resDir + '/log.csv')
        


INITIAL STATUS : 


,status,duration
model,,
SW_Pst27___Lx12Hx6Dx12__12,0,
SW_Pst04___Lx12Hx6Dx12__12,0,
SW_EfN2S_SpX16_Ptrd_AW1e3D1e3_Lx12Hx6Dx12_0193x0211x0094_12,0,
SW_EfN2S_SpX16_Ptrd_AW1e3D1e3_Lx12Hx6Dx12_ScFt_0193x0211x0094_12,0,
SW_EfN2S_SpX16_Ptrd_AW1e3D1e3_Lx12Hx6Dx12_ScBs_0193x0211x0094_12,0,



minDate :  2010-05-13 00:00:00
maxDate :  2023-04-19 00:00:00


  0%|          | 0/5 [00:00<?, ?it/s]

/Users/greg/Projects/sundl/notebooks/ficat/../../sundl/dataloader/sdocml.py:742: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ).apply(lambda x: x[-1]).shift(freq = f'-{offLabel}H')#[:-int(offLabel/2)]


...

/Users/greg/Projects/sundl/notebooks/ficat/../../sundl/dataloader/sdocml.py:205: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ).apply(lambda x: x[0])#[24*scalar_lag:]
/Users/greg/Projects/sundl/notebooks/ficat/../../sundl/dataloader/sdocml.py:205: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ).apply(lambda x: x[0])#[24*scalar_lag:]
/Users/greg/Projects/sundl/notebooks/ficat/../../sundl/dataloader/sdocml.py:205: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posit

------------------------------
labels.shape (266, 12)
------------------------------
3 incomplete training dates
3 incomplete val dates

MODEL PARAMETERS #: 20.00M
of which trainable #: tf.Tensor(4303, shape=(), dtype=int32)
Epoch 1/2
13/13 [==============================] - ETA: 0s - loss: 8468.0635 - mae: 268.7648 - maep: 63.6896 - rmse: 331.5070
Epoch 1: val_rmse did not improve from 85.00000
13/13 [==============================] - 45s 3s/step - loss: 8468.0635 - mae: 268.7648 - maep: 63.6896 - rmse: 331.5070 - val_loss: 10549.3252 - val_mae: 76.2502 - val_maep: 17.9408 - val_rmse: 102.7099
Epoch 2/2
13/13 [==============================] - ETA: 0s - loss: 8456.2402 - mae: 67.8243 - maep: 16.8960 - rmse: 91.9578
Epoch 2: val_rmse did not improve from 85.00000
13/13 [==============================] - 30s 2s/step - loss: 8456.2402 - mae: 67.8243 - maep: 16.8960 - rmse: 91.9578 - val_loss: 10552.5684 - val_mae: 76.2548 - val_maep: 17.9405 - val_rmse: 102.7257


100%|██████████| 1/1 [04:04<00:00, 244.69s/it]

Train  loss:  ['8.47e+03', '8.46e+03']
Train  mae:  ['268.76', '67.82']
Train  maep:  ['63.69', '16.90']
Train  rmse:  ['331.51', '91.96']

Val val_loss:  ['1.05e+04', '1.06e+04']
Val val_mae:  ['76.25', '76.25']
Val val_maep:  ['17.94', '17.94']
Val val_rmse:  ['102.71', '102.73']



100%|██████████| 5/5 [16:37<00:00, 199.46s/it]
